In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [4]:
# Leer csv con labels
DATA_PATH = "data/"
df = pd.read_csv(DATA_PATH + "labels.csv")

In [5]:
df.head(5)

,isic_id,malignant
0,ISIC_0015670,0.0
1,ISIC_0015845,0.0
2,ISIC_0015864,0.0
3,ISIC_0015902,0.0
4,ISIC_0024200,0.0


In [6]:
# Definición de tamaño conjunto de entrenamiento, validación y test
SPLIT_SIZE = 0.8
VAL_SIZE = 0.15
SEED = 2025 # Para hacer el split determinista. Tener reproducibilidad

In [8]:
# Crear datasets/ids de entrenamiento, validación y test
temp_df, test_df = train_test_split(
    df,
    train_size=SPLIT_SIZE,
    random_state=SEED,
    stratify=df["malignant"]
)

train_df, val_df = train_test_split(
    temp_df,
    train_size=(1 - VAL_SIZE),
    random_state=SEED,
    stratify=temp_df["malignant"]
)

In [9]:
# Guardar archivos
train_df.to_csv(DATA_PATH + "train_split.csv")
val_df.to_csv(DATA_PATH + "validation_split.csv")
test_df.to_csv(DATA_PATH + "test_split.csv")